In [1]:
import numpy as np
import pyvista as pv
from tqdm import tqdm
import torch
from multiprocessing import Pool
import os

In [13]:
def process_level(path: str):
    level = amulet.load_level(path)
    chunks = list(level.all_chunk_coords("minecraft:overworld"))

    blocks_list = []

    # Adding a progress bar to the chunks iteration
    for chunk_coords in tqdm(chunks, desc="Processing Chunks", unit="chunk"):
        chunk = level.get_chunk(chunk_coords[0], chunk_coords[1], "minecraft:overworld")
        
        # Adding a progress bar to the section iteration
        for section in chunk.blocks.sections:
            section_data = torch.from_numpy(chunk.blocks.get_section(section))
            non_zero_indices = torch.argwhere(section_data != 0)
            
            world_x = chunk_coords[0] * 16 + non_zero_indices[:, 0]
            world_y = section * 16 + non_zero_indices[:, 1]
            world_z = chunk_coords[1] * 16 + non_zero_indices[:, 2]
            
            block_coords = torch.column_stack((world_x, world_z, world_y))
            blocks_list.append(block_coords)

    # Convert the list of arrays into a single NumPy array
    blocks = torch.concatenate(blocks_list, axis=0) if blocks_list else torch.empty((0, 3), dtype=int)

    level.close()
    del level
    del chunks

    return blocks

In [ ]:
blocks = process_level('./custom_saves/train/1')
print(blocks.shape)

In [9]:
# Create base mesh
voxels = pv.PolyData()
size = 1.0

# Add cubes to mesh
for point in blocks:
    cube = pv.Cube(center=point, x_length=size, y_length=size, z_length=size)
    voxels += cube

# Create static plotter
plotter = pv.Plotter(notebook=True, off_screen=True)
plotter.add_mesh(voxels, color='#808080', opacity=1)

# Show static image
plotter.show(jupyter_backend='static')

KeyboardInterrupt: 